### In each data fold, there is a raw data subfolder and a syn data subfolder, which represent the raw data collection without synchronisation but with high precise timestep, and the synchronised data but without high precise timestep.


## Here is the header of the sensor file and ground truth file.

## vicon (vi*.csv)

Time  Header  translation.x translation.y translation.z rotation.x rotation.y rotation.z rotation.w

## Sensors (imu*.csv)

Time attitude_roll(radians) attitude_pitch(radians) attitude_yaw(radians) rotation_rate_x(radians/s) rotation_rate_y(radians/s) rotation_rate_z(radians/s) gravity_x(G) gravity_y(G) gravity_z(G) user_acc_x(G) user_acc_y(G) user_acc_z(G) magnetic_field_x(microteslas) magnetic_field_y(microteslas) magnetic_field_z(microteslas)

In [1]:
! pip install numpy.quaternion

In [2]:
import os
import time
from os import path as osp

import numpy as np
import torch
import json

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader

from data_Oxiod import *
from transformations import *
from metric import compute_ate_rte
from model_resnet1d import *
from pathlib import Path

/Users/andrewcheam/Documents/GitHub/untitled folder/IMU_localisation/Oxiod/source/transformations.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def change_cf(ori, vectors):
/Users/andrewcheam/Documents/GitHub/untitled folder/IMU_localisation/Oxiod/source/transformations.py:96: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def __call__(self, feature, targe

In [3]:
def get_dataset(root_dir, data_list):
    
    sequence = OxfordGlobSpeedSequence(osp.join(root_dir, data_list))
    return sequence.features


def get_dataset_from_list(root_dir, list_path, args, **kwargs):
    with open(list_path) as f:
        data_list = [s.strip().split(',' or ' ')[0] for s in f.readlines() if len(s) > 0 and s[0] != '#']
    return get_dataset(root_dir, data_list)

## Domain accelerations

In [5]:
fig, axes = plt.subplots(nrows=14, ncols=2, figsize=(12, 24))
dataset_folder = osp.join(Path(os.getcwd()).parents[2], 'localisation_datasets')


data_path_list = ['handbag/data1/combined1.csv', 'handbag/data1/combined2.csv', 'handheld/data1/combined1.csv', 'handheld/data1/combined2.csv', 'pocket/data1/combined1.csv', 'pocket/data1/combined2.csv',
                  'running/data1/combined1.csv', 'running/data1/combined2.csv', 'slow walking/data1/combined1.csv', 'slow walking/data1/combined2.csv',
                  'trolley/data1/combined1.csv', 'trolley/data1/combined2.csv', 'large scale/floor1/combined1.csv','large scale/floor1/combined2.csv']
#data_path_list = ['handbag/data1/combined1.csv']

# Access each subplot individually
for i in range(14):
    data_path = data_path_list[i]
    data = get_dataset(dataset_folder+'/Oxford Inertial Odometry Dataset/', data_path)
    acc_data = data[:,:3]
    direction = ['X','Y']
    for j in range(2):
        ax = axes[i, j]
        ax.plot(acc_data[3000:5000,j])  # Example plot on each subplot
        seq_name = data_path.split('/')[0]
        ax.set_title(seq_name + direction[j])

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

## Finding out error of accelerations

## This is to convert all the imu and ground truth data into combined files (Not in use now)

In [15]:
# Read the CSV file
def concatData(imu_file_path, gt_file_path, combined_file_path):
    gt_column_names = ['Time', 'Header', 'translation.x', 'translation.y', 'translation.z', 'rotation.x', 'rotation.y', 'rotation.z', 'rotation.w']

    imu_column_names = ['Time', 'attitude_roll(radians)', 'attitude_pitch(radians)', 'attitude_yaw(radians)',
                'rotation_rate_x(radians/s)', 'rotation_rate_y(radians/s)', 'rotation_rate_z(radians/s)',
                'gravity_x(G)', 'gravity_y(G)', 'gravity_z(G)', 'user_acc_x(G)', 'user_acc_y(G)', 'user_acc_z(G)',
                'magnetic_field_x(microteslas)', 'magnetic_field_y(microteslas)', 'magnetic_field_z(microteslas)']
    imu_data_syn = pd.read_csv(imu_file_path)
    gt_data_syn = pd.read_csv(gt_file_path)
    imu_data_syn.columns = imu_column_names
    gt_data_syn.columns = gt_column_names
    df1 = imu_data_syn[['rotation_rate_x(radians/s)', 'rotation_rate_y(radians/s)', 'rotation_rate_z(radians/s)',
                        'user_acc_x(G)', 'user_acc_y(G)', 'user_acc_z(G)', 'gravity_x(G)', 'gravity_y(G)', 'gravity_z(G)']]
    df2 = gt_data_syn[['translation.x', 'translation.y', 'translation.z', 'rotation.x', 'rotation.y', 'rotation.z', 'rotation.w']]

    sequence_data = pd.concat([df1,df2], axis = 1)
    sequence_data.to_csv(combined_file_path, index = False)
    print("SUCCESSFULLY COMBINED DATA")
    print("IMU FILE PATH: ", imu_file_path)
    print("GROUNDTRUTH FILE PATH: ", gt_file_path)
    print("COMBINED GROUND TRUTH", combined_file_path)
    print("=================================================================")
    print()
    

In [20]:
folder_path = '/home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset'  # Specify the folder path

# Iterate through all file paths in the folder
for root, dirs, files in os.walk(folder_path):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        for root1, dirs1, files1 in os.walk(dir_path):
            for dir_name1 in dirs1:
                dir_path1 = os.path.join(root1, dir_name1)
                for root2, dirs2, files2 in os.walk(dir_path1 + '/syn'):
                    # files2.sort()
                    # print(dir_path1)
                    # print(files2)
                    # print()
                    length = len(files2)
                    for i in range(int(length/2)):
                        print("i: ", i)
                        print("Length: ", length)
                        imu_file_path = os.path.join(root2, f'imu{i+1}.csv')
                        if os.path.exists(os.path.join(root2, f'vi{i+1}.csv')):
                            gt_file_path = os.path.join(root2, f'vi{i+1}.csv')
                        elif os.path.exists(os.path.join(root2, f'tango{i+1}.csv')):
                            gt_file_path = os.path.join(root2, f'tango{i+1}.csv')
                        output_file_path = os.path.join(dir_path1, f"combined{i+1}.csv")
                        concatData(imu_file_path, gt_file_path, output_file_path)

i:  0
Length:  12
SUCCESSFULLY COMBINED DATA
IMU FILE PATH:  /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/syn/imu1.csv
GROUNDTRUTH FILE PATH:  /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/syn/vi1.csv
COMBINED GROUND TRUTH /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/combined1.csv

i:  1
Length:  12
SUCCESSFULLY COMBINED DATA
IMU FILE PATH:  /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/syn/imu2.csv
GROUNDTRUTH FILE PATH:  /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/syn/vi2.csv
COMBINED GROUND TRUTH /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/combined2.csv

i:  2
Length:  12
SUCCESSFULLY COMBINED DATA
IMU FILE PATH:  /home/jovyan/localisation_datasets/Oxford Inertial Odometry Dataset/pocket/data2/syn/imu3.csv
GROUNDTRUTH FILE PATH:  /home/jovyan/localisation_datasets/Oxford